# Web Scrape labelled images of building styles

The ACOToronto website contains the TOBuilt database -- an open source database of images and information about buildings and structures in toronto.
http://www.acotoronto.ca/tobuilt_new_detailed.php

Order of actions
* we will first use the search tool on the main page to find all the available architectural styles (http://www.acotoronto.ca/tobuilt_new_detailed.php)
* then we will call the page for each style.  This page contains a thumbnail image of each building classified in that style, a link for more details and basic info about the building.  
Note: you generally have to follow a redirection to get to the style page
(http://www.acotoronto.ca/search_buildingsR-d.php?sid=8065)
* download the image locally to image/<style>
* call building details page to get info on building dates, architects etc
(http://www.acotoronto.ca/show_building.php?BuildingID=3883)
 page is structured with alternating building_info and building_info2 divs
building_info contains the name of the info to follow
building_info2 contains the value
The companies row contains the architects. There can be muliple architects for a building, so these are < li > items.
Sometimes the archtect is a hyperlink, but not always, so need to handle both cases specially

American colonial
Annex Style
Art deco
Arts and Crafts
Beaux arts
Brutalist
Byzantine
Chicago style
Classical revival
Commercial style
Contemporary
Deconstructivism
Dutch colonial
Early modern
Edwardian classical
English Cottage style
Georgian revival
Gothic revival
Greek revival
International style
Italianate
Late modernist
Log construction
Mid century expressionist
Mirrored tower
Modern classical
Modern historicist
Modernist
Neo Palladian
Neo-Chateau
Neo-Georgian
Neo-modernist
Neo-Tudor
Postmodern
Prairie style
Queen Anne
Regency
Renaissance revival
Richardsonian Romanesque
Romanesque revival
Sculptural
Second empire
Shingle style
Spanish colonial
Toronto Bay and Gable
Workers Cottage

In [12]:
# Import libaries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import string
import re
import urllib
import os
import time
from datetime import datetime

Load Page

In [13]:
def load_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Error connecting: status code {response.status_code}")

In [26]:
def download_image(style, img_url):
    '''
    * downloads image to the appropriate style sub-folder in the images directory (and creates folder if missing)
    * to test: download_image('test','/tobuilt_bk/php/Buildingimages/106BedfordRd.jpg')
    '''

    dest_dir = f"../Images/{style}"

    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    full_url = f'{site_root}/{img_url}'
    image_path = f"{dest_dir}/{img_url.split('/')[-1]}"
    res = urllib.request.urlretrieve(full_url, image_path)


#### Building Details pages
* page is structured with alternating building_info and building_info2 divs
* building_info contains the name of the info to follow
* building_info2 contains the value
* The companies row contains the architects.  There can be muliple architects for a building, so these are < li > items.
* Sometimes the archtect is a hyperlink, but not always, so need to handle both cases specially

In [27]:
def get_building_details(bld_link):
    
    flags_dict = {
        'Completed':None,
        'Demolished': None,
        'Companies': None,
        'Type':None,
        'Current use': None,
        'Heritage': None,
       'Notes': None
    }
    architects=[]
    
    full_url = f'{site_root}{bld_link}'
    #print(full_url)
    html_bld = load_page(full_url)
    soup_bld = BeautifulSoup(html_bld)
    # get all building info elements
    info = soup_bld.find_all('div',{'class': 'building_info'})
    for inf in info:
        for flag in flags_dict.keys():
            if flag in inf.text:
                if 'Companies' in flag:
                    company_box = inf.find_next()

                    if 'Architect - ' in company_box.text:
                        li_items = company_box.find_all('li')

                        for arc in li_items:
                            if (arc.find('span')):
                                architect = arc.find('span').previousSibling
                                if architect.name =='a':
                                    architects.append(architect.text.strip())
                                else:
                                    architects.append(architect.replace('-','').strip())
                                
                        flags_dict[flag] = architects
                else:
                    flags_dict[flag] = inf.find_next().text.strip()
    return flags_dict

### Get Buildings for style

In [28]:
def get_buildings_for_style(soup, style):
    buildings=[]
   
    for building in soup.find_all('div',{'class': 'box'}):
        build_style = building.find('div',{'class': 'box_image'})['style']
        image_url = re.findall('\((.*?)\)', build_style)[0]
        build_dict={
            'Style':style,
            'Name':building.findChild('span',{'class':'title'}).text,
            'Bld_link':building.findChild('a').get_attribute_list(key='href')[0],
            'Image':image_url,
            'Address': str(building.findChild('div',{'class':'the_box_text'}).findChild('p')).replace('<br/>',' ').replace('<p>','').replace('</p>','')
        }

        download_image(style,image_url)
        #follow link to get more info on building
        if debug==True: 
            print(build_dict['Bld_link'])
        build_details_dict = get_building_details(build_dict['Bld_link'])
        build_dict = {**build_dict, **build_details_dict}
        buildings.append(build_dict)
   # bld_df = pd.DataFrame(buildings)
    return buildings

In [36]:

def process_style (style):
    print (f"loading {style}")
    curr_style_url = f"{style_url}?MainStyle={style}"
    #print(curr_style_url)
    html = load_page(curr_style_url)
    soup = BeautifulSoup(html)
    if 'document.location' in soup.text:
        # need to redirect to another page
        redirect_url = soup.text.strip().replace('document.location = "','').replace('";',"")
        curr_style_url=f"{site_root}{redirect_url}"
       # print(curr_style_url)
        html2 = load_page(curr_style_url)
        soup = BeautifulSoup(html2)
        
    style_list = get_buildings_for_style(soup, style)
    print(style_list)
    buildings_list.extend(style_list)

### Set up base variables

In [29]:

main_page = 'http://www.acotoronto.ca/tobuilt_new_detailed.php'
style_url="http://www.acotoronto.ca/search_buildingsDB_d2.php"
site_root = "http://www.acotoronto.ca/"
debug=False
buildings_list=[]


In [31]:

html = load_page(main_page)
soup = BeautifulSoup(html)
style_options = soup.find('select',{'name':'MainStyle'}).find_all('option')
for style in style_options[1:]:
    process_style(style.text)    


American colonial
Annex Style
Art deco
Arts and Crafts
Beaux arts
Brutalist
Byzantine
Chicago style
Classical revival
Commercial style
Contemporary
Deconstructivism
Dutch colonial
Early modern
Edwardian classical
English Cottage style
Georgian revival
Gothic revival
Greek revival
International style
Italianate
Late modernist
Log construction
Mid century expressionist
Mirrored tower
Modern classical
Modern historicist
Modernist
Neo Palladian
Neo-Chateau
Neo-Georgian
Neo-modernist
Neo-Tudor
Postmodern
Prairie style
Queen Anne
Regency
Renaissance revival
Richardsonian Romanesque
Romanesque revival
Sculptural
Second empire
Shingle style
Spanish colonial
Toronto Bay and Gable
Workers Cottage
loading American colonial
[{'Style': 'American colonial', 'Name': 'Robert Grieg House', 'Bld_link': 'show_building.php?BuildingID=4386', 'Image': '/tobuilt_bk/php/Buildingimages/20ChestnutPark.jpg', 'Address': '20 Chestnut Park Rosedale Toronto', 'Completed': '1905', 'Demolished': None, 'Companies': ['A

http://www.acotoronto.ca/search_buildingsDB_d2.php?MainStyle=Postmodern
http://www.acotoronto.ca/search_buildingsDB_d2.php?MainStyle=Arts%20and%20Crafts
http://www.acotoronto.ca/search_buildingsDB_d2.php?MainStyle=Annex%20Style

In [33]:
bld_df = pd.DataFrame(buildings_list)

In [35]:
bld_df.to_csv('../data/aco_buildings_'+ str(round(time.time(),0)) + '.csv')

In [ ]:
#